<a href="https://colab.research.google.com/github/shakil1819/LLM-RAG-SCiRev/blob/main/LLM_RAG_SciRev_00.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 18 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.2 [186 kB]
Fetched 186 kB in 0s (2,150 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 120874 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.2_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.2) ...
Setting up poppler-utils (22.02.0-2ubuntu0.2) ...
Processing triggers for man-db (2.10.2-1) ...


In [2]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq langchain==0.0.173 --progress-bar off
!pip install -qqq chromadb==0.3.23 --progress-bar off
!pip install -qqq pypdf==3.8.1 --progress-bar off
!pip install -qqq pygpt4all==1.1.0 --progress-bar off
!pip install -qqq pdf2image==1.16.3 --progress-bar off

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
tensorflow 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.8.0 which is incompatible.
  Preparing metadata (setup.py) ... done


In [4]:
!pip install llmhub

In [5]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import GPT4All
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from pdf2image import convert_from_path

In [6]:
!wget https://gpt4all.io/models/ggml-gpt4all-j-v1.3-groovy.bin

--2023-10-22 07:06:37--  https://gpt4all.io/models/ggml-gpt4all-j-v1.3-groovy.bin
Resolving gpt4all.io (gpt4all.io)... 104.26.1.159, 172.67.71.169, 104.26.0.159, ...
Connecting to gpt4all.io (gpt4all.io)|104.26.1.159|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3785248281 (3.5G)
Saving to: ‘ggml-gpt4all-j-v1.3-groovy.bin’

ggml-gpt4all-j-v1.3 100%[===================>]   3.52G  25.3MB/s    in 2m 1s   

2023-10-22 07:08:39 (29.8 MB/s) - ‘ggml-gpt4all-j-v1.3-groovy.bin’ saved [3785248281/3785248281]



In [7]:
images = convert_from_path("/content/drive/MyDrive/Colab Notebooks/~Pictures/2023279689.pdf", dpi=88)
len(images)

6

In [8]:
loader = PyPDFLoader("/content/drive/MyDrive/Colab Notebooks/~Pictures/2023279689.pdf")

In [9]:
documents = loader.load_and_split()

In [10]:
len(documents)

12

In [11]:
print(documents[0].page_content)

CycleGAN-Based Data Augmentation with CNN
and Vision Transformers (ViT) Models for
Improved Maize Leaf Disease Classification
Syed Taha Yeasin Ramadan
Department of Computer Science and Engineering
Military Institute of Science and Technology (MIST)
Dhaka, Bangladesh
tahayeasin11@gmail.comTanjim Sakib
Department of Computer Science and Engineering
Military Institute of Science and Technology (MIST)
Dhaka, Bangladesh
tsakib77@gmail.com
Md. Ahsan Rahat
Department of Computer Science and Engineering
Military Institute of Science and Technology (MIST)
Dhaka, Bangladesh
ahsanrahat11@gmail.comShakil Mosharrof
Department of Computer Science and Engineering
Military Institute of Science and Technology (MIST)
Dhaka, Bangladesh
shakilmrf8@gmail.com
Abstract —Crop losses pose a serious danger to global food
security, and this problem also affects maize crops. To success-
fully address this issue, precise disease detection techniques
are required. However, a major hurdle to developing reliable
mod

In [12]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=256, chunk_overlap=16)
texts = text_splitter.split_documents(documents)

In [13]:
len(texts)

144

In [14]:
print(texts[0].page_content)

CycleGAN-Based Data Augmentation with CNN
and Vision Transformers (ViT) Models for
Improved Maize Leaf Disease Classification
Syed Taha Yeasin Ramadan
Department of Computer Science and Engineering
Military Institute of Science and Technology (MIST)


In [15]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [16]:
db = Chroma.from_documents(texts, embeddings, persist_directory="db")

In [18]:
model_n_ctx = 1000
model_path = "./ggml-gpt4all-j-v1.3-groovy.bin"
llm = GPT4All(model=model_path, n_ctx=1000, backend="gptj", verbose=False)

In [19]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True,
    verbose=False,
)

In [ ]:
%%time
res = qa(
    "Your Task is to generate literature review.To generate a suitable prompt for extracting a literature review from a research paper, you can use the following format: In their study, [author name] et al. proposed [main outcomes of the research], achieving [best metrics obtained from the paper]. However, the researchers also acknowledged [limitations mentioned by the researcher]. This work contributes to the field by addressing [research gap].' - Here, you would replace [author name],[main outcomes of the research], [best metrics obtained from the paper], [limitations mentioned by the researcher], and [research gap] with specific information from the research paper you are interested in. This format provides a structured way to request the relevant information you need from the paper."
)

In [ ]:
res

In [ ]:
print(res["Literature review : "])
